# Libraries

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split

import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split, LeaveOneOut

import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Autoencoder function

In [2]:

def build_autoencoder(input_shape=(256, 256, 1)):
   
    input_img = Input(shape=input_shape, name='input_image')


    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Flatten(name='encoded_features')(x)


    radius_pred = Dense(1, activation='linear', name='radius_output')(encoded)


    x = Dense(1024, activation='relu')(encoded)
    x = Reshape((4, 4, 64))(x)


    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)                # 8x8
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)                # 16x16
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)                # 64x64
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)                # 128x128
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)                # 256x256
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='image_output')(x)

    
    autoencoder = Model(inputs=input_img, outputs=[decoded, radius_pred], name='autoencoder')




  
    autoencoder = Model(inputs=input_img, outputs=[decoded, radius_pred], name='autoencoder')
    return autoencoder


def train_autoencoder(autoencoder, x_train, radii_train, x_val, radii_val, epochs=50, batch_size=32):
    autoencoder.compile(
        optimizer='adam',
        loss={'image_output': 'mse', 'radius_output': 'mse'}
    )

   
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

   
    history = autoencoder.fit(
        x_train,
        {'image_output': x_train, 'radius_output': radii_train},
        validation_data=(x_val, {'image_output': x_val, 'radius_output': radii_val}),
        epochs=50,
        batch_size=32
    )


    autoencoder.summary()

    return history



# LeaveOneOut

In [5]:
df = pd.read_excel('../combined_results.xlsx')

df['Image Path'] = df['Image Path'].apply(lambda x: '.' + x)
image_paths = df['Image Path'].tolist()  
img_height, img_width = 256, 256  
images = []

for path in image_paths:
  
    img = load_img(path, target_size=(img_height, img_width), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0  
    images.append(img_array)


radii = df['circle_radius'].values   
ccol = df['center_y(ccol)'].values
crow = df['center_x(crow)'].values

images = np.array(images)  # Shape: (num_samples, img_height, img_width, 1)
radii = radii.reshape(-1, 1)  # Shape: (num_samples, 1)
ccol = ccol.reshape(-1, 1)
crow = crow.reshape(-1, 1)

loo=LeaveOneOut()

for i, (train_index, test_index) in enumerate(loo.split(df)):
    # print(f"Fold {i}:")

    # print(f"  Train: index={train_index}")

    # print(f"  Test:  index={test_index}")
    # images_train = [images[j] for j in train_index]
    images_train = images[train_index]
    images_test=images[test_index]

    # radii_train = [radii[j] for j in train_index]
    radii_train = radii[train_index]
    radii_test=radii[test_index]

    autoencoder = build_autoencoder(input_shape=(img_height, img_width, 1))

    autoencoder.compile(
        optimizer='adam',
        loss={
            'image_output': 'mse',
            'radius_output': 'mse'
        }
    )

    # print(images_test.shape)  # Should be (num_samples, 256, 256, 1)
    # print(radii_train.shape)   # Should be (num_samples, 1)
    # print(ccol_train.shape)    # Should be (num_samples, 1)
    # print(crow_train.shape)    # Should be (num_samples, 1)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = autoencoder.fit(
        images_train,
        {
            'image_output': images_train, 
            'radius_output': radii_train
        },
        validation_data=(images_test,{
            'image_output': images_test,
            'radius_output': radii_test
        }),
        validation_split=0.1,
        epochs=50,
        # batch_size=32,
        batch_size=1,
        callbacks=[early_stopping],
        verbose=1
    )

    # Save the model for the current fold
    model_path = os.path.join('models', f'autoencoder_fold_{i+1}.h5')
    autoencoder.save(model_path)
    print(f"Saved model to {model_path}")

Epoch 1/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - image_output_loss: 0.0366 - loss: 94.7723 - radius_output_loss: 94.7356 - val_image_output_loss: 0.0082 - val_loss: 4.3745 - val_radius_output_loss: 4.3663
Epoch 2/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - image_output_loss: 0.0182 - loss: 4.9094 - radius_output_loss: 4.8912 - val_image_output_loss: 0.0076 - val_loss: 1.9952 - val_radius_output_loss: 1.9876
Epoch 3/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 425s 2s/step - image_output_loss: 0.0156 - loss: 1.6635 - radius_output_loss: 1.6479 - val_image_output_loss: 0.0064 - val_loss: 1.9232 - val_radius_output_loss: 1.9168
Epoch 4/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - image_output_loss: 0.0132 - loss: 0.9418 - radius_output_loss: 0.9286 - val_image_output_loss: 0.0078 - val_loss: 0.9375 - val_radius_output_loss: 0.9297
Epoch 5/50
221/221 ━━━━━━━━━━━━━━━━━━━━ 421s 2s/step - image_output_loss: 0.0066 - loss: 0.5025 - radius_output_loss: 0.4959 - val_image_output_loss: 0.0055 - val

KeyboardInterrupt: 